In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (19575 Collins Ave UNIT 17, Sunny Isles Beach,...
2     (2201 Collins Ave, Miami Beach, FL 33139, USA,...
3     (17749 Collins Ave APT 801, Sunny Isles Beach,...
4     (300 South Pointe Dr APT 1105, Miami Beach, FL...
5     (700 NE 25th St, Miami, FL 33137, USA, (25.800...
6      (Miami Beach, FL, USA, (25.790654, -80.1300455))
7     (881 Ocean Dr #11h, Key Biscayne, FL 33149, US...
8     (650 NE 32nd St, Miami, FL 33137, USA, (25.806...
9     (6365 Collins Ave APT 3303, Miami Beach, FL 33...
10    (3581 E Glencoe St #408, Miami, FL 33133, USA,...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Regalia Condo\r19575 Collins Ave 17\rSunny Isl...,6/2/2022,8/30/2022,8,"$10,700,000.00",4992,"$2,143.43",Jill Hertzberg,Coldwell Banker Realty,Eric Grabois,Envision Realty Group,Regalia Condo 19575 Collins Ave 17 Sunny Isles...,Regalia Condo,19575 Collins Ave 17 Sunny Isles Beach,"(19575 Collins Ave UNIT 17, Sunny Isles Beach,...","(25.956929, -80.1193868, 0.0)",25.956929,-80.119387,0.0
2,W Resort & Residence\r2201 Collins Ave PH28\rM...,11/8/2021,9/1/2022,254,"$8,750,000.00",2642,"$3,311.88",Anna Sherrill,One Sotheby's Int'l Realty,Eloy Carmenate,The Corcoran Group,W Resort & Residence 2201 Collins Ave PH28 Mia...,W Resort & Residence,2201 Collins Ave PH28 Miami Beach,"(2201 Collins Ave, Miami Beach, FL 33139, USA,...","(25.7979013, -80.1275924, 0.0)",25.797901,-80.127592,0.0
3,Mansions at Acqualina\r17749 Collins Ave 801\r...,2/16/2021,8/31/2022,505,"$6,250,000.00",4609,"$1,356.04",Denise Rubin,Coldwell Banker Realty,Denise Rubin,Coldwell Banker Realty,Mansions at Acqualina 17749 Collins Ave 801 Su...,Mansions at Acqualina,17749 Collins Ave 801 Sunny Isles Beach,"(17749 Collins Ave APT 801, Sunny Isles Beach,...","(25.9400518, -80.12056079999999, 0.0)",25.940052,-80.120561,0.0
4,Portofino Tower\r300 S Pointe Dr 1105\rMiami ...,5/29/2022,9/1/2022,45,"$4,300,000.00",2670,"$1,825.90",Dora Puig,Luxe Living Realty,Samantha Scalzo,Interealty Exchange Inc,Portofino Tower 300 S Pointe Dr 1105 Miami Beach,Portofino Tower,300 S Pointe Dr 1105 Miami Beach,"(300 South Pointe Dr APT 1105, Miami Beach, FL...","(25.7677332, -80.1349941, 0.0)",25.767733,-80.134994,0.0
5,Star Lofts on the Bay\r700 NE 25th St PH1\rMiami,6/17/2022,9/1/2022,35,"$2,930,000.00",4965,$590.13,Daniel Tzinker,The Agency Florida LLC,Astrid Miniet,Robert Slack LLC,Star Lofts on the Bay 700 NE 25th St PH1 Miami,Star Lofts on the Bay,700 NE 25th St PH1 Miami,"(700 NE 25th St, Miami, FL 33137, USA, (25.800...","(25.8008285, -80.1857847, 0.0)",25.800829,-80.185785,0.0
6,Monad Terrace\r1300 Monad Ter 10C\rMiami Beach,12/20/2021,8/31/2022,159,"$2,850,000.00",1453,"$1,961.46",Raymond Jacomo,Douglas Elliman,Ryan M Thorburn,Urbaniza Realty,Monad Terrace 1300 Monad Ter 10C Miami Beach,Monad Terrace,1300 Monad Ter 10C Miami Beach,"(Miami Beach, FL, USA, (25.790654, -80.1300455))","(25.790654, -80.1300455, 0.0)",25.790654,-80.130045,0.0
7,Casa Del Mar Condo\r881 Ocean Dr 11H\rKey Bisc...,11/22/2021,8/30/2022,210,"$2,650,000.00",3120,$849.36,Juan Velasquez,Douglas Elliman,Angela Santamaria PA,The Corcoran Group,Casa Del Mar Condo 881 Ocean Dr 11H Key Biscayne,Casa Del Mar Condo,881 Ocean Dr 11H Key Biscayne,"(881 Ocean Dr #11h, Key Biscayne, FL 33149, US...","(25.685686, -80.15729309999999, 0.0)",25.685686,-80.157293,0.0
8,Paraiso Bay\r650 NE 32nd St PH5004\rMiami,6/14/2022,9/2/2022,78,"$2,500,000.00",3599,$785.67,Ellen Manas,One Sotheby's Int'l Realty,Andrew Becerra Jr. PA,EXP Realty LLC,Paraiso Bay 650 NE 32nd St PH5004 Miami,Paraiso Bay,650 NE 32nd St PH5004 Miami,"(650 NE 32nd St, Miami, FL 33137, USA, (25.806...","(25.8068747, -80.1862424, 0.0)",25.806875,-80.186242,0.0
9,Akoya Condo\r6365 Collins Ave 3303\rMiami Beach,6/21/2022,8/29/2022,26,"$2,150,000.00",1761,"$1,220.90",Nabil Jimenez,Douglas Elliman,Stefania Cambarau,Douglas Elliman,Akoya Condo 6365 Collins Ave 3303 Miami Beach,Akoya Condo,6365 Collins Ave 3303 Miami Beach,"(6365 Collins Ave APT 3303, Miami Beach, FL 33...","(25.8460388, -80.1196447, 0.0)",25.846039,-80.119645,0.0
10,T0he Fairchild\r3581 E Glencoe St 408\rMiami,5/12/2022,8/31/2022,86,"$2,100,000.00",2003,"$1,048.43",Arline Silver,Brown Harris Stevens,Frank Giordano,Fortune International Realty,T0he Fairchild 3581 E Glencoe St 408 Miami,T,0he Fairchild 3581 E Glencoe St 408 Miami,"(3581 E Glencoe St #408, Miami, FL 33133, USA,...","(25.7401466, -80.21740

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [14]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [10]:
df.at[10,'building_name']=('The Fairchild')
df.at[10,'address_only']=('3581 E Glencoe Street unit 408')

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [13]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('August 28th - September 3rd')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [14]:
m.save('index.html')